In [30]:
import pandas as pd
import numpy as np

# Load the datasets
fomc_df = pd.read_csv("FOMC_dataset.csv")
spx_df = pd.read_csv("^SPX_data.csv")
nasdaq100_df = pd.read_csv("^NDX_data.csv")
russell2000_df = pd.read_csv("^RUT_data.csv")
dowjones_df = pd.read_csv("^DJI_data.csv")
gdelt_df = pd.read_csv("gdelt/GDELT_dataset.csv")
cameo_df = pd.read_csv("gdelt/CAMEO_eventcodes.txt", sep="\t")
# Display the first few rows of each dataset to understand their structure
# fomc_df.head(), spx_df.head()

C:\Users\danie\AppData\Local\Temp\ipykernel_19972\3656949642.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  gdelt_df = pd.read_csv("gdelt/GDELT_dataset.csv")


In [ ]:
def calculate_changes(df):
    # Explicitly convert 'Date' to datetime and handle errors
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    
    # Drop rows where 'Date' is NaT after conversion
    df = df.dropna(subset=['Date']).reset_index(drop=True)
    
    # Explicitly ensure 'Close' is numeric, handle errors
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    df = df.dropna(subset=['Close']).reset_index(drop=True)

    # Ensure data is sorted
    df.sort_values(by='Date', inplace=True)

    # Calculate Daily Close Change %
    df['Daily Close Chg%'] = df['Close'].pct_change() * 100

    # Monthly Close Change % (last close of previous month vs last close of current month)
    df['Month'] = df['Date'].dt.to_period('M')
    # Get the last close of each month
    last_monthly_close = df.groupby('Month')['Close'].last()
    # Map last close to each row and shift to get previous month's last close
    df['Prev Month Last Close'] = df['Month'].map(last_monthly_close).shift(1)
    # Current month's last close for each row
    df['Current Month Last Close'] = df['Month'].map(last_monthly_close)
    # Calculate monthly change
    monthly_chg = (df['Current Month Last Close'] - df['Prev Month Last Close']) / df['Prev Month Last Close'] * 100
    # Only record on the last day of each month
    df['Monthly Close Chg%'] = monthly_chg.where(df['Date'].dt.is_month_end, other=float('nan'))

    # Yearly Close Change % (last close of previous year vs last close of current year)
    df['Year'] = df['Date'].dt.year
    # Get the last close of each year
    last_yearly_close = df.groupby('Year')['Close'].last()
    # Map last close to each row and shift to get previous year's last close
    df['Prev Year Last Close'] = df['Year'].map(last_yearly_close).shift(1)
    # Current year's last close for each row
    df['Current Year Last Close'] = df['Year'].map(last_yearly_close)
    # Calculate yearly change
    yearly_chg = (df['Current Year Last Close'] - df['Prev Year Last Close']) / df['Prev Year Last Close'] * 100
    # Only record on the last day of each year
    df['Year Close Chg%'] = yearly_chg.where(df['Date'].dt.is_year_end, other=float('nan'))

    # Drop helper columns
    df.drop(columns=['Month', 'Year', 'Prev Month Last Close', 'Current Month Last Close', 
                     'Prev Year Last Close', 'Current Year Last Close'], inplace=True)

    return df

In [32]:
cameo_df.head()
# check for missing values in cameo_df
cameo_df.isnull().sum()

CAMEOEVENTCODE      0
EVENTDESCRIPTION    0
dtype: int64

In [33]:
gdelt_df.head()

,event_date,EventCode,event_count,average_tone,average_goldstein,total_mentions,total_sources,total_articles
0,2024-12-31,144,5,-2.697527,-7.5,35,5,35
1,2024-12-31,195,20,-6.151181,-10.0,102,22,102
2,2024-12-31,193,598,-5.894597,-10.0,2752,619,2657
3,2024-12-31,043,2450,-0.541621,2.8,10865,2518,10598
4,2024-12-31,154,18,-1.224078,-7.2,114,18,114


In [34]:
gdelt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676092 entries, 0 to 676091
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   event_date         676092 non-null  object 
 1   EventCode          676092 non-null  object 
 2   event_count        676092 non-null  int64  
 3   average_tone       676092 non-null  float64
 4   average_goldstein  675650 non-null  float64
 5   total_mentions     676092 non-null  int64  
 6   total_sources      676092 non-null  int64  
 7   total_articles     676092 non-null  int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 41.3+ MB


In [35]:
# remove null values in average_goldstein
gdelt_df['event_date'] = pd.to_datetime(gdelt_df['event_date'])
gdelt_df = gdelt_df[gdelt_df['average_goldstein'].notnull()]
gdelt_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 675650 entries, 0 to 676091
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   event_date         675650 non-null  datetime64[ns]
 1   EventCode          675650 non-null  object        
 2   event_count        675650 non-null  int64         
 3   average_tone       675650 non-null  float64       
 4   average_goldstein  675650 non-null  float64       
 5   total_mentions     675650 non-null  int64         
 6   total_sources      675650 non-null  int64         
 7   total_articles     675650 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 46.4+ MB


In [36]:
# Convert Date columns to datetime format
fomc_df['Date'] = pd.to_datetime(fomc_df['Date'])
fomc_df['Year'] = fomc_df['Date'].dt.year
fomc_df['Month'] = fomc_df['Date'].dt.month
spx_df = spx_df.iloc[2:].reset_index(drop=True)  # Remove metadata rows
nasdaq100_df = nasdaq100_df.iloc[2:].reset_index(drop=True)
russell2000_df = russell2000_df.iloc[2:].reset_index(drop=True)
djia_df = dowjones_df.iloc[2:].reset_index(drop=True)

# Rename columns for clarity
spx_df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Name']
spx_df = spx_df.drop(columns=['Name'])  # Remove unnecessary column
nasdaq100_df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Name']
nasdaq100_df = nasdaq100_df.drop(columns=['Name'])
russell2000_df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Name']
russell2000_df = russell2000_df.drop(columns=['Name'])
djia_df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Name']
djia_df = djia_df.drop(columns=['Name'])

# Convert numeric columns to float type
numeric_cols = ['Close', 'High', 'Low', 'Open', 'Volume']
spx_df[numeric_cols] = spx_df[numeric_cols].astype(float)
nasdaq100_df[numeric_cols] = nasdaq100_df[numeric_cols].astype(float)
russell2000_df[numeric_cols] = russell2000_df[numeric_cols].astype(float)
djia_df[numeric_cols] = djia_df[numeric_cols].astype(float)

# Remove records before a certain cutoff date
cutoff_date = pd.to_datetime("2013-01-01")
fomc_df = fomc_df[fomc_df['Date'] >= cutoff_date]

# Group by Date and aggregate
numeric_cols = fomc_df.select_dtypes(include=[np.number]).columns.drop('FOMC Meeting', errors='ignore') 
fomc_df = fomc_df.groupby('Date')[numeric_cols].mean().reset_index()

# Fill missing FOMC Meeting with 0 (no meeting)
print("Missing Values:\n", fomc_df.isnull().sum())

# Cell 6: Treat Outliers
fomc_df['Real GDP'] = fomc_df['Real GDP'].replace(28630.739, np.nan)
fomc_df['Real GDP'] = fomc_df['Real GDP'].interpolate(method='linear')
fomc_df['Home Sales'] = fomc_df['Home Sales'].clip(lower=400, upper=800) 

# Cell 8: Select Key Columns for Tableau
key_columns = [
    'Date', 'Year', 'Month', 'CPI', 'Unemployment', 'Fed Rate', 'Real GDP', 
    'Inflation', 'Home Sales', 'Retail Sales', 'FOMC Meeting', 'Regime',
    'Industry Production Lag_1', 'Unemployment Lag_1', 'YoY CPI', 'YoY Unemployment',
    'Month_Sin', 'Month_Cos'
]

fomc_df = fomc_df[[col for col in key_columns if col in fomc_df.columns]]
print("Final Columns:", fomc_df.columns.tolist())



Missing Values:
 Date                    0
Unnamed: 0              0
CPI                     0
Industry Production     0
PCE                     0
                       ..
Unemployment Lag_12     0
Wage Increase Lag_12    0
Home Sales Lag_12       0
Retail Trade Lag_12     0
Real GDP Lag_12         0
Length: 102, dtype: int64
Final Columns: ['Date', 'Year', 'Month', 'CPI', 'Unemployment', 'Fed Rate', 'Real GDP', 'Inflation', 'Home Sales', 'Retail Sales', 'Industry Production Lag_1', 'Unemployment Lag_1', 'YoY Unemployment', 'Month_Sin', 'Month_Cos']


In [37]:
spx_df = calculate_changes(spx_df)
nasdaq100_df = calculate_changes(nasdaq100_df)
russell2000_df = calculate_changes(russell2000_df)
djia_df = calculate_changes(djia_df) 

In [38]:
fomc_df.to_csv("FOMC_dataset_cleaned.csv", index=False)
spx_df.to_csv("SPX_dataset_cleaned.csv", index=False)
nasdaq100_df.to_csv("NDX_dataset_cleaned.csv", index=False)
russell2000_df.to_csv("RUT_dataset_cleaned.csv", index=False)
djia_df.to_csv("DJI_dataset_cleaned.csv", index=False)
gdelt_df.to_csv("GDELT_dataset_cleaned.csv", index=False)